# Pendulum
## The notebook describes a solution of the problem of controlling a pendulum to be raised to a vertical position and held in this state using a on-policy first-visit Monte Carlo control method 

### Workspace
Workspace is divided into 4 values, the cartesian coordinate system x and y, theta angle in radians and tau the torque of the pendulum


### Action Space
Force applied to the pendulum in the range of continuous values from -2.0 to 2.0

### Observation Space
Observation Space contains 3 values, the x and y coordinates, and the angular velocity of the pendulum.
x = cos(theta) continuous value from -1.0 to 1.0
y = sin(theta) continuous value from -1.0 to 1.0
Angular velocity is continuous value from -8.0 to 8.0


### I've made those changes:
Converted observation space from 3 values to 2, by obtaining theta angle from its cosine and sine using the x and n values I know

### I used the built-in award function which is described as:
### r = -(theta^2^ + 0.1 * theta_dt^2^ + 0.001 * torque^2^)

## My solution is written based on this pseudocode
![Pseudo code](./mc_onpolicy.jpg)

In [1]:
import gymnasium as gym
import numpy as np
from matplotlib import pyplot as plt

PRELOAD = True

In [6]:
# env = gym.make('Pendulum-v1', render_mode="human", max_episode_steps=600); env.metadata['render_fps'] = 60
env = gym.make('Pendulum-v1', max_episode_steps=600)

count_of_episodes = 500000
discount_rate = 0.95

if not PRELOAD:
    epsilon = 1.0
    epsilon_min = 0.1
    epsilon_decay = 0.9999
else:
    epsilon = 0.15
    epsilon_min = 0.05
    epsilon_decay = 0.9999

total_reward = 0

action_space_size = 41
""" count of action [-2 : 2] with 0.1 step, 0->19 = [-2 : -0.1], 20 = 0, 21->40 = [0.1 : 2] """
observation_space_size = [63, 161]
"""
First is count of pendulum state [-pi : pi] with 0.1 step, 0->31 = [-pi : -pi/32], 32 = 0, 33->62 = [pi/32 : pi]
Second is count of pendulum angular velocity [-8 : 8] with 0.1 step, 0->79 = [-8 : -0.1], 80 = 0, 81->160 = [0.1 : 8]
"""

total_reward_for_episode = list()

In [ ]:
action_space = np.linspace(-2, 2, num=action_space_size)
observation_space = [np.linspace(-np.pi, np.pi, num=observation_space_size[0]),
                     np.linspace(-8.0, 8.0, num=observation_space_size[1])]
if not PRELOAD:
    q_table = np.random.uniform(low=-1, high=1, size=(observation_space_size + [action_space_size]))
    # q_table = np.zeros(observation_space_size + [action_space_size])
    returns_table = np.zeros(observation_space_size + [action_space_size])
    returns_table_count = np.zeros(observation_space_size + [action_space_size])
    policy = np.ones((observation_space_size + [action_space_size])) / action_space_size
else:
    q_table = np.load("D:/Programming/MachineLearning2/Zadanie1/MC_q_table_pendulum.npy")
    policy = np.load("D:/Programming/MachineLearning2/Zadanie1/policy_table.npy")
    returns_tables = np.load("D:/Programming/MachineLearning2/Zadanie1/returns_tables.npy")
    returns_table = returns_tables[0]
    returns_table_count = returns_tables[1]

In [ ]:
def get_indexes(state):
    theta = np.arctan2(state[0], state[1])
    index_state = np.digitize(theta, observation_space[0]) - 1
    index_velocity = np.digitize(state[2], observation_space[1]) - 1
    return index_state, index_velocity

In [ ]:
def choose_action(index_state, index_velocity):
    action_probabilities = policy[index_state, index_velocity]
    action = np.random.choice(np.arange(action_space_size), p=action_probabilities)
    return action

In [ ]:
for episode in range(1, count_of_episodes+1):
    state = env.reset()
    state = state[0]
    episode_history = list()
    done = False
    episode_reward = 0

    while not done:
        index_state, index_velocity = get_indexes(state)

        if np.random.random() > epsilon:
            # action = np.argmax(get_discrete_action(state))
            action = choose_action(index_state, index_velocity)
        else:
            action = np.random.randint(0, action_space_size)

        step = [action_space[action]]
        observation, reward, done, truncated, *info = env.step(step)

        total_reward += reward
        episode_reward += reward

        # new_index_state, new_index_velocity = get_indexes(observation)

        episode_history.append(((index_state, index_velocity), action, reward))

        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

        if truncated:
            break

    G = 0
    for i in reversed(range(0, len(episode_history))):
        
        this_episode = episode_history[i]
        indexes, action, reward = this_episode
        G = discount_rate * G + reward

        if (indexes, action) not in episode_history[:-1]:
            # index_state, index_velocity = indexes
            returns_table[indexes[0], indexes[1], action] += G
            returns_table_count[indexes[0], indexes[1], action] += 1
            q_table[indexes[0], indexes[1], action] = \
                returns_table[indexes[0], indexes[1], action] /\
                returns_table_count[indexes[0], indexes[1], action]

            best_action_index = np.argmax(q_table[indexes[0], indexes[1]])
            for a in range(action_space_size):
                if a == best_action_index:
                    policy[indexes[0], indexes[1], a] = 1 - epsilon + (epsilon / action_space_size)
                else:
                    policy[indexes[0], indexes[1], a] = epsilon / action_space_size

    if episode % 1000 == 0:
        print("Episode: {}".format(episode))
        # print("Total reward = {}".format(total_reward))
        print("Average reward = {}\n".format(total_reward/episode))

        returns_tables = [returns_table, returns_table_count]
        np.save("D:/Programming/MachineLearning2/Zadanie1/MC_q_table_pendulum", q_table)
        np.save("D:/Programming/MachineLearning2/Zadanie1/returns_tables", returns_tables)
        np.save("D:/Programming/MachineLearning2/Zadanie1/policy_table", policy)

    total_reward_for_episode.append(episode_reward)

In [ ]:
plt.plot(total_reward_for_episode)
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.show()

In [ ]:
returns_tables = [returns_table, returns_table_count]
np.save("D:/Programming/MachineLearning2/Zadanie1/MC_q_table_pendulum", q_table)
np.save("D:/Programming/MachineLearning2/Zadanie1/returns_tables", returns_tables)
np.save("D:/Programming/MachineLearning2/Zadanie1/policy_table", policy)

# TESTING

In [8]:
env = gym.make('Pendulum-v1', max_episode_steps=600)

count_of_episodes = 50000
discount_rate = 0.95

epsilon = 1.0
epsilon_min = 0.1
epsilon_decay = 0.9999


total_reward = 0

action_space_size = 21
""" count of action [-2 : 2] with 0.1 step, 0->19 = [-2 : -0.1], 20 = 0, 21->40 = [0.1 : 2] """
observation_space_size = [31, 81]
"""
First is count of pendulum state [-pi : pi] with 0.1 step, 0->31 = [-pi : -pi/32], 32 = 0, 33->62 = [pi/32 : pi]
Second is count of pendulum angular velocity [-8 : 8] with 0.1 step, 0->79 = [-8 : -0.1], 80 = 0, 81->160 = [0.1 : 8]
"""

total_reward_for_episode = list()
action_space = np.linspace(-2, 2, num=action_space_size)
observation_space = [np.linspace(-np.pi, np.pi, num=observation_space_size[0]),
                     np.linspace(-8.0, 8.0, num=observation_space_size[1])]

q_table = np.random.uniform(low=-1, high=1, size=(observation_space_size + [action_space_size]))
# q_table = np.zeros(observation_space_size + [action_space_size])
returns_table = np.zeros(observation_space_size + [action_space_size])
returns_table_count = np.zeros(observation_space_size + [action_space_size])
policy = np.ones((observation_space_size + [action_space_size])) / action_space_size

def get_indexes(state):
    theta = np.arctan2(state[0], state[1])
    index_state = np.digitize(theta, observation_space[0]) - 1
    index_velocity = np.digitize(state[2], observation_space[1]) - 1
    return index_state, index_velocity


def choose_action(index_state, index_velocity):
    action_probabilities = policy[index_state, index_velocity]
    action = np.random.choice(np.arange(action_space_size), p=action_probabilities)
    return action


for episode in range(1, count_of_episodes+1):
    state = env.reset()
    state = state[0]
    episode_history = list()
    done = False
    episode_reward = 0

    while not done:
        index_state, index_velocity = get_indexes(state)

        if np.random.random() > epsilon:
            # action = np.argmax(get_discrete_action(state))
            action = choose_action(index_state, index_velocity)
        else:
            action = np.random.randint(0, action_space_size)

        step = [action_space[action]]
        observation, reward, done, truncated, *info = env.step(step)

        total_reward += reward
        episode_reward += reward

        # new_index_state, new_index_velocity = get_indexes(observation)

        episode_history.append(((index_state, index_velocity), action, reward))

        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

        if truncated:
            break

    G = 0
    for i in reversed(range(0, len(episode_history))):
        # reward = episode_history[i][-1]

        this_episode = episode_history[i]
        indexes, action, reward = this_episode
        G = discount_rate * G + reward

        if (indexes, action) not in episode_history[:-1]:
            # index_state, index_velocity = indexes
            returns_table[indexes[0], indexes[1], action] += G
            returns_table_count[indexes[0], indexes[1], action] += 1
            q_table[indexes[0], indexes[1], action] = \
                returns_table[indexes[0], indexes[1], action] /\
                returns_table_count[indexes[0], indexes[1], action]

            best_action_index = np.argmax(q_table[indexes[0], indexes[1]])
            for a in range(action_space_size):
                if a == best_action_index:
                    policy[indexes[0], indexes[1], a] = 1 - epsilon + (epsilon / action_space_size)
                else:
                    policy[indexes[0], indexes[1], a] = epsilon / action_space_size

    total_reward_for_episode.append(episode_reward)
    
    if episode % 10000 == 0:
        print("Episode: {}".format(episode))
        print("Total reward = {}".format(total_reward))
        print("Average reward = {}\n".format(total_reward/episode))
    
    
print("First episode reward: {}".format(total_reward_for_episode[0]))
print("Last episode reward: {}".format(total_reward_for_episode[-1]))


Episode: 10000
Total reward = -37263544.787386894
Average reward = -3726.3544787386895
Episode: 20000
Total reward = -73316519.6250489
Average reward = -3665.825981252445
Episode: 30000
Total reward = -109187917.38633704
Average reward = -3639.5972462112345
Episode: 40000
Total reward = -144906881.40441257
Average reward = -3622.672035110314
Episode: 50000
Total reward = -180479409.51723495
Average reward = -3609.588190344699

First episode reward: -4936.852030573195
Last episode reward: -4812.688742901135


## Test data in tabular form

| Episodes | DR   | Avg. reward | Total reward    | action space size | observation space size | First episode reward | Last episode reward |
|-------|---|------------|-----------------|---|------------------------|---------------------|---------------------|
| 10000 | 0.9 | -3914      | -39145430       | 41 | 63, 161                | -3452               |                     |
| 20000 | 0.9 | -3823      | -76474999       | 41 | 63, 161                |
| 30000 | 0.9 | -3759      | -112790082      | 41 | 63, 161                |
| 40000 | 0.9 | -3721      | -148847811      | 41 | 63, 161                |
| 50000 | 0.9 | -3698      | -184915135      | 41 | 63, 161                |                     | -3491               |

| Episodes | DR   | Avg. reward | Total reward    | action space size | observation space size | First episode reward | Last episode reward |
|-------|---|-------------|-----------------|-------------------|------------------------|----------------------|---------------------|
| 10000 | 0.9 | -3712       | -37122246       | 21                | 31, 81                 | -3029                |                     |
| 20000 | 0.9 | -3648       | -72965356       | 21                | 31, 81                 |
| 30000 | 0.9 | -3625       | -108773386      | 21                | 31, 81                 |
| 40000 | 0.9 | -3611       | -144442189      | 21                | 31, 81                 |
| 50000 | 0.9 | -3602       | -180112652      | 21                | 31, 81                 |                      | -3111                |

| Episodes | DR   | Avg. reward | Total reward | action space size | observation space size | First episode reward | Last episode reward |
|-------|------|-------------|-------------|-------------------|------------------------|----------------------|---------------------|
| 10000 | 0.95 | -3906       | -39069204   | 41                | 63, 161                | -4275                |                     |
| 20000 | 0.95 | -3819       | -76394531   | 41                | 63, 161                |
| 30000 | 0.95 | -3760       | -112807625  | 41                | 63, 161                |
| 40000 | 0.95 | -3725       | -149016157  | 41                | 63, 161                |
| 50000 | 0.95 | -3698       | -184924282  | 41                | 63, 161                |                      | -4006               |

| Episodes | DR   | Avg. reward | Total reward | action space size | observation space size | First episode reward | Last episode reward |
|-------|------|-------------|-------------|-------------------|------------------------|----------------------|---------------------|
| 10000 | 0.95 | -3726       | -37263544   | 21                | 31, 81                 | -4936                |                     |
| 20000 | 0.95 | -3665       | -73316519   | 21                | 31, 81                 |
| 30000 | 0.95 | -3639       | -109187917  | 21                | 31, 81                 |
| 40000 | 0.95 | -3622       | -144906881  | 21                | 31, 81                 |
| 50000 | 0.95 | -3609       | -180479409  | 21                | 31, 81                 |                      | -4812               |